## makemore: becoming a backprop ninja

In [ ]:
# there no change change in the first several cells from last lecture

In [53]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [54]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [55]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [56]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
# ok biolerplate done, now we get to the action:

In [106]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [107]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP
block_size = 3

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [108]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [204]:
bnmeani.shape

torch.Size([1, 64])

In [227]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.7614, grad_fn=<NegBackward0>)

In [228]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------
# loss = -(a+b+c..)/n
# da = -1/n
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

# logprobs = log(probs)
# dprobs = 1/probs
dprobs = (1 / probs) * dlogprobs

# probs[32 x 27] = counts[32 x 27] * counts_sum_inv[32 x 1]
# two operations: replication and multiplication
# backprop: multiplication and squashing by taking sum of the derivatives

# c = a*b
# a[3x3] * b[3x1]
# a11*b1 a12*b1 a13*b1
# a21*b2 a22*b2 a23*b2
# a31*b3 a32*b3 a33*b3

# dc/db = a --> [3x3]
# dL/db = dL/dc [3x3] * dc/db [3x3] --> sum over 2nd dim

# ref- micrograd: if one value's output is used in two expressions,
#      we take sum of the derivatives during backprop
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

# counts_sum_inv = counts_sum**-1
dcounts_sum = -counts_sum**-2 * dcounts_sum_inv

# probs = counts * counts_sum_inv
dcounts = counts_sum_inv * dprobs # broadcast
# counts_sum = counts.sum(1, keepdims=True)
dcounts += torch.ones_like(counts) * dcounts_sum # broadcast

# counts = norm_logits.exp()
dnorm_logits = counts * dcounts

# norm_logits[32x27] = logits[32x27] - logit_maxes[32x1]
dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdim=True)

# logit_maxes = logits.max(1, keepdim=True).values
dlogits = torch.zeros_like(logits)
dlogits[range(n), logits.max(1).indices] = 1 
dlogits *= dlogit_maxes
# norm_logits = logits - logit_maxes
dlogits += dnorm_logits.clone()

# logits = h @ W2 + b2 
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0, keepdim=True)

# h = torch.tanh(hpreact)
dhpreact = (1-h**2) * dh

# hpreact = bngain * bnraw + bnbias
dbngain = (bnraw * dhpreact).sum(0) # these are squashing operations
dbnbias = (1 * dhpreact).sum(0) # these are squashing operations
dbnraw = bngain * dhpreact

# bnraw = bndiff * bnvar_inv
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw

# bnvar_inv = (bnvar + 1e-5)**-0.5
dbnvar = -0.5 * (bnvar + 1e-5)**-1.5 * dbnvar_inv

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
dbndiff2 = torch.ones_like(bndiff2) * (1/(n-1)) * dbnvar

# bndiff2 = bndiff**2
dbndiff += 2 * bndiff * dbndiff2.clone()

# bndiff = hprebn - bnmeani
dbnmeani = (-1 * dbndiff).sum(0, keepdim=True)
dhprebn = dbndiff.clone()

# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
dhprebn += torch.ones_like(hprebn) * (1/n) * dbnmeani

# hprebn = embcat @ W1 + b1 
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0, keepdim=True)

# embcat = emb.view(emb.shape[0], -1)
demb = dembcat.view(emb.shape)

# emb = C[Xb]
# for each character in the input, increase the gradient of C by demb of that character
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
        
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [229]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.7613778114318848 diff: -4.76837158203125e-07


In [230]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits *= 1/n
# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [202]:
p.shape

torch.Size([32, 27])

In [164]:
logits

tensor([[ 8.8804e-01,  1.0511e+00, -4.5256e-01,  3.9286e-01, -4.4437e-01,
          1.0129e+00, -2.2095e-01,  1.0465e-01, -5.8544e-01,  1.4312e-03,
          1.7337e-01,  1.7488e-01,  8.1199e-02, -2.2635e-01,  7.6980e-02,
         -8.6776e-01, -1.2193e+00, -4.6882e-01, -7.1514e-01,  4.9330e-01,
          3.9535e-01, -3.0296e-01, -3.2465e-01,  8.1970e-01,  6.5088e-01,
         -1.5624e-01, -5.0913e-01],
        [ 2.8221e-01,  3.9474e-01,  8.8000e-01,  2.6190e-01, -5.8928e-02,
         -7.2980e-02, -8.0004e-01,  8.5076e-02, -6.6819e-01, -4.6135e-01,
          2.4005e-01,  1.4374e-01,  1.0129e-01, -3.5946e-01,  7.6927e-02,
         -1.6044e-01, -3.7348e-01, -9.0309e-01, -6.2636e-01, -1.0963e-01,
         -6.8351e-01, -4.9715e-01, -1.0194e+00,  5.6306e-01, -4.6535e-01,
         -5.9723e-02, -5.9850e-01],
        [-4.1014e-01, -2.9998e-01, -7.0495e-01, -9.5547e-01, -3.8487e-01,
          3.3247e-01,  5.3231e-01,  5.4157e-01,  5.2048e-01, -1.0155e-01,
         -4.3883e-01,  5.7445e-02,  2.43

In [165]:
Yb

tensor([ 8, 14, 15, 22,  0, 19,  9, 14,  5,  1, 20,  3,  8, 14, 12,  0, 11,  0,
        26,  9, 25,  0,  1,  1,  7, 18,  9,  3,  5,  9,  0, 18])

In [166]:
logits[range(n), Yb] #picks up the logits of actual output from each example

tensor([-0.5854,  0.0769, -0.3090, -0.0397, -0.5783, -0.1864,  0.2458, -0.4822,
         0.3411, -0.7542,  0.5559,  1.9058,  0.5611,  0.3866,  0.3109,  0.1643,
        -0.3955,  0.3152, -0.2857, -0.0594,  0.5521,  0.4835, -0.8361, -0.6060,
        -0.0916,  0.5580,  0.3234, -0.6375,  0.8020,  0.0093,  0.3244,  0.2942],
       grad_fn=<IndexBackward0>)

In [170]:
logits[range(n), Yb].shape

torch.Size([32])

In [171]:
dlogits[range(n), Yb].shape

torch.Size([32])

In [175]:
logits.exp().sum(1).shape

torch.Size([32])

In [233]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [234]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [210]:
temp = (bngain/n) * ((n-1)*(1e-5) - hprebn.var(0, keepdim=True)) / (hprebn.var(0,keepdim=True) + 1e-5)**1.5
temp.shape

torch.Size([1, 64])

In [247]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

    # kick off optimization
    for i in range(max_steps):

      # minibatch construct
      ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
      Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

      # forward pass
      emb = C[Xb] # embed the characters into vectors
      embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
      # Linear layer
      hprebn = embcat @ W1 + b1 # hidden layer pre-activation
      # BatchNorm layer
      # -------------------------------------------------------------
      bnmean = hprebn.mean(0, keepdim=True)
      bnvar = hprebn.var(0, keepdim=True, unbiased=True)
      bnvar_inv = (bnvar + 1e-5)**-0.5
      bnraw = (hprebn - bnmean) * bnvar_inv
      hpreact = bngain * bnraw + bnbias
      # -------------------------------------------------------------
      # Non-linearity
      h = torch.tanh(hpreact) # hidden layer
      logits = h @ W2 + b2 # output layer
      loss = F.cross_entropy(logits, Yb) # loss function

      # backward pass
      for p in parameters:
        p.grad = None
      #loss.backward() # use this for correctness comparisons, delete it later!

      # manual backprop! #swole_doge_meme
      # -----------------
      # YOUR CODE HERE :)
      dlogits = F.softmax(logits, 1)
      dlogits[range(n), Yb] -= 1
      dlogits *= 1/n

      # logits = h @ W2 + b2 
      dh = dlogits @ W2.T
      dW2 = h.T @ dlogits
      db2 = dlogits.sum(0)

      # h = torch.tanh(hpreact)
      dhpreact = (1-h**2) * dh

      # hpreact = bngain * bnraw + bnbias
      dbngain = (bnraw * dhpreact).sum(0, keepdim=True) # these are squashing operations
      dbnbias = (1 * dhpreact).sum(0, keepdim=True) # these are squashing operations
      dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

      # hprebn = embcat @ W1 + b1 
      dembcat = dhprebn @ W1.T
      dW1 = embcat.T @ dhprebn
      db1 = dhprebn.sum(0)

      # embcat = emb.view(emb.shape[0], -1)
      demb = dembcat.view(emb.shape)

      # emb = C[Xb]
      # for each character in the input, increase the gradient of C by demb of that character
      dC = torch.zeros_like(C)
      for k in range(Xb.shape[0]):
        for j in range(Xb.shape[1]):
            ix = Xb[k,j]
            dC[ix] += demb[k,j]
      grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
      # -----------------
    #   for p, grad in zip(parameters, grads):
    #     print(p.grad.shape)
    #     print(grad.shape)
    #     print("-----------")


      # update
      lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
      for p, grad in zip(parameters, grads):
        #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
        p.data += -lr * grad # new way of swole doge TODO: enable

      # track stats
      if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
      lossi.append(loss.log10().item())
  

12297
      0/ 200000: 3.7743
  10000/ 200000: 2.1729
  20000/ 200000: 2.4344
  30000/ 200000: 2.4682
  40000/ 200000: 1.9937
  50000/ 200000: 2.3230
  60000/ 200000: 2.3803
  70000/ 200000: 2.0618
  80000/ 200000: 2.3880
  90000/ 200000: 2.1701
 100000/ 200000: 1.9276
 110000/ 200000: 2.3165
 120000/ 200000: 1.9830
 130000/ 200000: 2.4268
 140000/ 200000: 2.4048
 150000/ 200000: 2.1954
 160000/ 200000: 1.9155
 170000/ 200000: 1.8480
 180000/ 200000: 1.9497
 190000/ 200000: 1.8970


In [246]:
# useful for checking your gradients
for p,g in zip(parameters, grads):
  cmp(str(tuple(p.shape)), g, p)

(27, 10)        | exact: False | approximate: True  | maxdiff: 2.60770320892334e-08
(30, 200)       | exact: False | approximate: True  | maxdiff: 1.1175870895385742e-08
(200,)          | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09
(200, 27)       | exact: False | approximate: True  | maxdiff: 2.2351741790771484e-08
(27,)           | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
(1, 200)        | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
(1, 200)        | exact: False | approximate: True  | maxdiff: 4.6566128730773926e-09


In [248]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [249]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.073286771774292
val 2.1111414432525635


In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [250]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

mora.
mayah.
seel.
ndhayla.
reimani.
jarlee.
adered.
elin.
shi.
jenleigh.
sana.
arleigh.
malaia.
noshubergihiman.
trick.
welle.
joseulanu.
zayven.
jamyleyeks.
kayshaston.
